In [48]:
import sys
import os
sys.path.insert(1, '..')

import numpy as np
import scipy as sp
import pandas as pd
from utils import *

import librosa
import soundfile as sf

import matplotlib.pyplot as plt
import seaborn as sns

import IPython.display as ipd

In [49]:
fp = '../dados/falas_04/resampled_rate/'
falas = os.listdir(fp)
caminhos_falas = [fp + fala for fala in falas]

fp = '../dados/ruidos_04//resampled_rate/'
ruidos = os.listdir(fp)
caminhos_ruidos = [fp + ruido for ruido in ruidos]

## Preprocess all the data

In [57]:
snrs = [-20, -10, -5, 0, 10, 20, 50]

speech_log = []
noise_log = []
noise_cut_log = []
snr_log = []
actual_snr_log = []
path_log = []
signal_number = 0
path = '../dados/preprocessed_signals/'

for caminho_fala, speech_data in zip(caminhos_falas, falas):
    fala , _ = librosa.load(caminho_fala, sr = None)
    
    for caminho_ruido, noise_data in zip(caminhos_ruidos, ruidos):
        ruido, _ = librosa.load(caminho_ruido, sr = None)
        
        for noise_cut in range(5):
            
            for snr in snrs:
                
                # Cut noise and pad speech
                recorte_ruido, cut_point = cut_noise(fala, ruido)
                speech_padded = zero_pad_beggining(recorte_ruido, fala)
                
                # Fix noise SNR
                normal_snr = snr_db(fala, recorte_ruido[16000:])
                fixed_noise = fix_snr(recorte_ruido, normal_snr, snr)
                
                # Noise signal and calculate actual snr
                noised_signal = fixed_noise + speech_padded
                real_snr = snr_db(speech_padded[16000:], fixed_noise[16000:])
                
                signal_name = 'signal_' + str(signal_number)
                
                # Log data from the signal
                speech_log.append(speech_data)
                noise_log.append(noise_data)
                noise_cut_log.append(cut_point)
                snr_log.append(snr)
                actual_snr_log.append(real_snr)
                path_log.append(signal_name)
               
                sf.write(path + signal_name +'.wav',
                         data = noised_signal,
                         samplerate = 8000)
                
                signal_number += 1

In [58]:
# Put all the data on a table
speech_table_data = zip(speech_log,
                        noise_log,
                        noise_cut_log,
                        snr_log,
                        actual_snr_log,
                        path_log)

cols = ['SIGNAL', 'NOISE', 'NOISE_CUT', 'EXPECTED_SNR', 'REAL_SNR', 'SIGNAL_NAME']

tabela_sinais = pd.DataFrame(data = speech_table_data,
                             columns = cols)

In [59]:
tabela_sinais

,SIGNAL,NOISE,NOISE_CUT,EXPECTED_SNR,REAL_SNR,SIGNAL_NAME
0,F180902.WAV,MIXER_HAND_MANUAL-mi#10CFB.flac,37116,-20,-2.000000e+01,signal_0
1,F180902.WAV,MIXER_HAND_MANUAL-mi#10CFB.flac,121404,-10,-1.000000e+01,signal_1
2,F180902.WAV,MIXER_HAND_MANUAL-mi#10CFB.flac,55870,-5,-5.000001e+00,signal_2
3,F180902.WAV,MIXER_HAND_MANUAL-mi#10CFB.flac,24885,0,-5.047750e-07,signal_3
4,F180902.WAV,MIXER_HAND_MANUAL-mi#10CFB.flac,179835,10,1.000000e+01,signal_4
...,...,...,...,...,...,...
3145,F181007.WAV,MIX MASTER-start-run-stop.flac,43339,-5,-5.000000e+00,signal_3145
3146,F181007.WAV,MIX MASTER-start-run-stop.flac,84687,0,1.665712e-07,signal_3146
3147,F181007.WAV,MIX MASTER-start-run-stop.flac,113949,10,1.000000e+01,signal_3147
3148,F181007.WAV,MIX MASTER-start-run-stop.flac,68716,20,2.000000e+01,signal_3148


In [60]:
tabela_sinais.to_csv('../dados/tabela_sinais.csv', index = False)